# Tests per agegroup
Data extracted from weekly report of RIVM

In [1]:
import pandas as pd
import tabula
import requests
from bs4 import BeautifulSoup
url = 'https://www.rivm.nl/coronavirus-covid-19/actueel/wekelijkse-update-epidemiologische-situatie-covid-19-in-nederland'
response = requests.get(url)

In [2]:
href = []
period = []
name = []
datetime = []
filename = []

def columns(l):
    href.append(l)
    p = pd.Period(l[-l.count('')+22:-l.count('')+29],"M")
    period.append(p)
    n = l[-l.count('')+30:-4]
    name.append(n)
    d = pd.to_datetime(l[-l.count('')+65:-l.count('')+73], format="%Y%m%d")
    datetime.append(d)
    f = l[-l.count('')+65:]
    filename.append(f)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser') 
aa = soup.find_all('a')
for a in aa:
    link = a.get('href')
    if link == None:
        pass
    elif link.endswith('.pdf'):
        columns(link)

pdfs = pd.DataFrame()
pdfs['period'] = period
pdfs['datetime'] = datetime
pdfs['name'] = name
pdfs['filename'] = filename
pdfs['href'] = href
pdfs['url'] = 'https://www.rivm.nl'+pdfs['href']
pdfs.head()

,period,datetime,name,filename,href,url
0,2021-03,2021-03-09,COVID-19_WebSite_rapport_wekelijks_20210309_1259,20210309_1259.pdf,/sites/default/files/2021-03/COVID-19_WebSite_...,https://www.rivm.nl/sites/default/files/2021-0...
1,2021-03,2021-03-02,COVID-19_WebSite_rapport_wekelijks_20210302_11...,20210302_1122_final_0.pdf,/sites/default/files/2021-03/COVID-19_WebSite_...,https://www.rivm.nl/sites/default/files/2021-0...
2,2021-02,2021-02-23,COVID-19_WebSite_rapport_wekelijks_20210223_12...,20210223_1223_final.pdf,/sites/default/files/2021-02/COVID-19_WebSite_...,https://www.rivm.nl/sites/default/files/2021-0...
3,2021-02,2021-02-16,COVID-19_WebSite_rapport_wekelijks_20210216_1423,20210216_1423.pdf,/sites/default/files/2021-02/COVID-19_WebSite_...,https://www.rivm.nl/sites/default/files/2021-0...
4,2021-02,2021-02-09,COVID-19_WebSite_rapport_wekelijks_20210209_11...,20210209_1115_final.pdf,/sites/default/files/2021-02/COVID-19_WebSite_...,https://www.rivm.nl/sites/default/files/2021-0...


In [4]:
rivm_pdf_data_loc = './data/pdfs/'

for row in pdfs.itertuples():
    r = requests.get(row.url, stream=True)
    with open(rivm_pdf_data_loc+row.filename, 'wb') as f:
        f.write(r.content)

## What can be used from the reports

### Chapter 4 agegroups
Reported, Hospitalised, Deceased agegroups

### Chapter 7 or 9
SARS-CoV-2 testen afgenomen door de GGD’en


In [7]:
## Find pagenumber of chapter
import pypdf2
import re

pdf = rivm_pdf_data_loc+pdfs.loc[0,'filename']
print(pdf)

# Open the pdf file
object = PyPDF2.PdfFileReader(r+pdf)

# Get number of pages
NumPages = object.getNumPages()

# Enter code here
String = "SARS-CoV-2 testen afgenomen door de GGD"

# Extract text and do the search
for i in range(0, NumPages):
    PageObj = object.getPage(i)
    Text = PageObj.extractText()
    if re.search(String,Text):
         print("Pattern Found on Page: " + str(i))

ModuleNotFoundError: No module named 'pypdf2'

In [95]:
tab = tabula.read_pdf(pdfs.url[0], pages=37, guess=True, stream=True)
df = tab[0]
#pd.DataFrame(df)

Got stderr: Mar 16, 2021 1:42:42 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Mar 16, 2021 1:42:42 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [96]:
df

,Unnamed: 0,Vanaf 30 november Afgelopen kalenderweek1
0,Leeftijdsgroep,Aantal Aantal Percentage Aantal Aantal Percentage
1,NaN,getest met positief positief getest positief p...
2,NaN,uitslag
3,0 - 3,16818 1693 10.1 3850 252 6.5
4,4 - 12,229506 26729 11.6 53425 2914 5.5
5,13 - 17,262396 39106 14.9 16443 1681 10.2
6,18 - 24,544296 65980 12.1 40122 4200 10.5
7,25 - 29,406738 39627 9.7 28462 2250 7.9
8,30 - 39,828809 71192 8.6 69130 4189 6.1
9,40 - 49,671575 74704 11.1 52025 4011 7.7


In [53]:
n = 0
for row in pdfs.itertuples():
    n=+1
    if n == 1:
        df = tabula.read_pdf(row.url, pages=29, guess=True, stream=True)
    else:
        temp = tabula.read_pdf(row.url, pages=29, guess=True, stream=True)
        df.append(temp)
print(df)

JavaNotFoundError: `java` command is not found from this Python process.Please ensure Java is installed and PATH is set for `java`